<a href="https://colab.research.google.com/github/superlearnerG/cs-self-learning/blob/master/NLPproject2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import time
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import torch.nn as nn
from transformers import BertModel
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# 这个函数能读取一个 .txt 文档，并将这个文档的第 1,4,7,10,... 等行号是3的倍数加1的行依次存放在列表texts中，第 2,5,8,11,... 等行号是3的倍数加2的行一依次存放在列表entities中，第 3,6,9,12,... 等行号被三整除的行依次存放在列表labels中
 
def read_txt_file(file_path, has_label):
    texts = []
    entities = []
    labels = []
    if has_label:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for i in range(len(lines)):
                lines[i] = lines[i].lower()
                if (i+1) % 3 == 1:
                    texts.append(lines[i].strip())
                elif (i+1) % 3 == 2:
                    entities.append(lines[i].strip())
                    # 把 $t$ 使用目标对象填充
                    texts[-1] = texts[-1].replace('$t$', entities[-1])
                else:
                    labels.append(eval(lines[i].strip()))
        return texts, entities, labels
    else:
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for i in range(len(lines)):
                lines[i] = lines[i].lower()
                if (i+1) % 3 == 1:
                    texts.append(lines[i].strip())
                elif (i+1) % 3 == 2:
                    entities.append(lines[i].strip())
                    # 把 $t$ 使用目标对象填充
                    texts[-1] = texts[-1].replace('$t$', entities[-1])
        return texts, entities, labels


texts, entities, labels = read_txt_file("train.txt", 1)
test_texts, test_entities, test_labels = read_txt_file("test.txt", 0)
labels = [(i+2) for i in labels]
print(texts)
print(entities)
print(labels)
print(test_texts)
print(test_entities)
print(test_labels)


# train_texts 存放了文本，train_entities 存放了每个文本中的目标对象，train_labels 存放了对应目标对象的情感极性。请使用 bert mini 训练一个预测目标对象的情感极性的模型


['but the staff was so horrible to us .', "to be completely fair , the only redeeming factor was the food , which was above average , but could n't make up for all the other deficiencies of teodora .", "the food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .", "the food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .", "the food is uniformly exceptional , with a very capable kitchen which will proudly whip up whatever you feel like eating , whether it 's on the menu or not .", "not only was the food outstanding , but the little ` perks ' were great .", "not only was the food outstanding , but the little ' perks ' were great .", 'our agreed favorite is the orrechiete with sausage and chicken -lrb- usually the waiters are kind enough to split the dish in half so you get to sample both mea

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00


In [3]:
# 分割出训练集和验证集
print(len(texts), len(entities), len(labels))
train_texts, eval_texts, train_entities, eval_entities, train_labels, eval_labels = train_test_split(texts, entities, labels, test_size=0.1)
print(len(train_texts), len(train_entities), len(train_labels), len(eval_texts), len(eval_entities), len(eval_labels))


3602 3602 3602
3241 3241 3241 361 361 361


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [5]:
# 加载bert的tokenize方法
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_encoded_comment = [tokenizer.encode(sent, add_special_tokens=True) for sent in train_texts]
eval_encoded_comment = [tokenizer.encode(sent, add_special_tokens=True) for sent in eval_texts]
'''
train_max_len = max([len(sent) for sent in train_encoded_comment])
print(train_max_len)
test_max_len = max([len(sent) for sent in test_encoded_comment])
print(test_max_len)
if train_max_len < test_max_len:
    print("train < test")
    MAX_LEN = test_max_len
else:
    print("train >= test")
    MAX_LEN = train_max_len
print(MAX_LEN)
'''
MAX_LEN = 128

In [6]:
'''
def preprocessing_for_bert(data, target_word):
    # 空列表来储存信息
    input_ids = []
    attention_masks = []

    # 每个句子循环一次
    for sent in data:
        # 将目标词转换为ID
        target_word_id = tokenizer.convert_tokens_to_ids(target_word)
        # 找到目标词在句子中的位置
        target_word_index = sent.index(target_word)
        # 将目标词ID插入到输入ID序列中的适当位置
        input_ids_sent = sent[:target_word_index] + target_word_id + sent[target_word_index+1:]
        encoded_sent = tokenizer.encode_plus(
            text=input_ids_sent,  # 预处理语句
            add_special_tokens=True,  # 加 [CLS] 和 [SEP]
            max_length=MAX_LEN,  # 截断或者填充的最大长度
            padding='max_length',  # 填充为最大长度
            return_attention_mask=True  # 返回 attention mask
        )

        # 把输出加到列表里面
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # 把list转换为tensor
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks
'''

def preprocessing_for_bert(texts, targets):
    input_ids = []
    attention_masks = []

    for i in range(len(texts)):
        # 将目标词转换为 ID
        target_id = tokenizer.convert_tokens_to_ids(targets[i])
        # 找到目标词在句子中的位置
        target_index = texts[i].index(targets[i])
        # 将目标词 ID 插入到输入 ID 序列中的适当位置
        #sentence = texts[i][:target_index] + target_id + texts[i][target_index+1:]
        sentence = "{} {} {}".format(texts[i][:target_index], target_id, texts[i][target_index+1:])
        encoded_sentence = tokenizer.encode_plus(
            text=sentence,
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )

        # 把输出加到列表里面
        input_ids.append(encoded_sentence.get('input_ids'))
        attention_masks.append(encoded_sentence.get('attention_mask'))

    # 把list转换为tensor
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks



In [7]:
# 在train，test上运行 preprocessing_for_bert 转化为指定输入格式
train_inputs, train_masks = preprocessing_for_bert(train_texts, train_entities)
eval_inputs, eval_masks = preprocessing_for_bert(eval_texts, eval_entities)

# 转化为tensor类型
train_labels = torch.tensor(train_labels)
eval_labels = torch.tensor(eval_labels)

# 用于BERT微调, batch size 16 or 32较好.
BATCH_SIZE = 32

# 给训练集创建 DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)
# print(train_dataloader)
# 给验证集创建 DataLoader
# 给训练集创建 DataLoader
eval_data = TensorDataset(eval_inputs, eval_masks, eval_labels)
eval_sampler = RandomSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=BATCH_SIZE)


In [8]:
"""关键点来了，开始训练！
Create BertClassifier

BERT base由12个transformer层组成，每个transformer层接收一系列token embedding
           注:token embedding就是文本的转化，embedding，transformer用的是随机生成然后训练
 
并在输出上生成相同数量的具有相同hidden size（或dim）的embedding。
[CLS] token最后transformer层的输出用作序列的特征来feed classifier

注意文本是512的限制，所以不能太大，我就是通过筛选排列得到的数据集
"""


# 自己定义的Bert分类器的类，微调Bert模型
class BertClassifier(nn.Module):
    def __init__(self, ):
        """
        freeze_bert (bool): 设置是否进行微调，0就是不，1就是调
        """
        super(BertClassifier, self).__init__()
        # 输入维度(hidden size of Bert)默认768，分类器隐藏维度，输出维度(label)
        D_in, H, D_out = 768, 100, 5

        # 实体化Bert模型
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # 实体化一个单层前馈分类器，说白了就是最后要输出的时候搞个全连接层
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),  # 全连接
            nn.ReLU(),  # 激活函数
            nn.Linear(H, D_out)  # 全连接
        )

    def forward(self, input_ids, attention_mask):
        # 开始搭建整个网络了
        # 输入
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        # 为分类任务提取标记[CLS]的最后隐藏状态，因为要连接传到全连接层去
        last_hidden_state_cls = outputs[0][:, 0, :]
        # 全连接，计算，输出label
        logits = self.classifier(last_hidden_state_cls)

        return logits


# 注意这个地方的logits是全连接的返回， 两个output就是01二分类，我们这里用的是ouput为3，就是老师所需要的三分类问题



In [9]:
"""
然后就是深度学习的老一套定义优化器还有学习率等
"""



def initialize_model(epochs=2):
    """
    初始化我们的bert，优化器还有学习率，epochs就是训练次数
    """
    # 初始化我们的Bert分类器
    print("initializing classifier!")
    bert_classifier = BertClassifier()
    print("to device!")
    # 用GPU运算
    bert_classifier.to(device)
    # 创建优化器
    print("initializing optimizer!")
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,  # 默认学习率
                      eps=1e-8  # 默认精度
                      )
    # 训练的总步数
    print("initializing total steps!")
    total_steps = len(train_dataloader) * epochs
    # 学习率预热
    print("initializing scheduler!")
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,  # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler


# 实体化loss function
loss_fn = nn.CrossEntropyLoss()  # 交叉熵

In [13]:
# 训练模型
def train(model, train_dataloader, test_dataloader=None, epochs=2, evaluation=False):
    # 开始训练循环
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # 表头
        print(f"{'Epoch':^7} | {'每40个Batch':^9} | {'训练集 Loss':^12} | {'测试集 Loss':^10} | {'测试集准确率':^9} | {'时间':^9}")
        print("-" * 80)

        # 测量每个epoch经过的时间
        t0_epoch, t0_batch = time.time(), time.time()

        # 在每个epoch开始时重置跟踪变量
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # 把model放到训练模式
        model.train()

        # 分batch训练
        for step, batch in enumerate(train_dataloader):
            batch_counts += 1
            # 把batch加载到GPU
            #print("batch to device!")
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
            #print(b_labels.shape)
            # 归零导数
            #print("zero deriv!")
            model.zero_grad()
            # 真正的训练
            #print("logits training!")
            logits = model(b_input_ids, b_attn_mask)
            #print(logits.shape)
            # 计算loss并且累加
            #print("cal loss!")
            loss = loss_fn(logits, b_labels)
            #print("add loss!")
            batch_loss += loss.item()
            total_loss += loss.item()
            # 反向传播
            #print("backward!")
            loss.backward()
            # 归一化，防止梯度爆炸
            #print("clip grad norm!")
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # 更新参数和学习率
            #print("step!")
            optimizer.step()
            scheduler.step()

            # Print每40个batch的loss和time
            if (step % 40 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # 计算40个batch的时间
                time_elapsed = time.time() - t0_batch

                # Print训练结果
                print(
                    f"{epoch_i + 1:^7} | {step:^10} | {batch_loss / batch_counts:^14.6f} | {'-':^12} | {'-':^13} | {time_elapsed:^9.2f}")

                # 重置batch参数
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # 计算平均loss 这个是训练集的loss
        avg_train_loss = total_loss / len(train_dataloader)

        print("-" * 80)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation:  # 这个evalution是我们自己给的，用来判断是否需要我们汇总评估
            # 每个epoch之后评估一下性能
            # 在我们的验证集/测试集上.
            test_loss, test_accuracy = evaluate(model, test_dataloader)
            # Print 整个训练集的耗时
            time_elapsed = time.time() - t0_epoch

            print(
                f"{epoch_i + 1:^7} | {'-':^10} | {avg_train_loss:^14.6f} | {test_loss:^12.6f} | {test_accuracy:^12.2f}% | {time_elapsed:^9.2f}")
            print("-" * 80)
        print("\n")


In [14]:
# 在测试集上面来看看我们的训练效果
def evaluate(model, test_dataloader):
    """
    在每个epoch后验证集上评估model性能
    """
    # model放入评估模式
    model.eval()

    # 准确率和误差
    test_accuracy = []
    test_loss = []

    # 验证集上的每个batch
    for batch in test_dataloader:
        # 放到GPU上
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # 计算结果，不计算梯度
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)  # 放到model里面去跑，返回验证集的ouput就是一行三列的
            # label向量可能性，这个时候还没有归一化所以还不能说是可能性，反正归一化之后最大的就是了

        # 计算误差
        loss = loss_fn(logits, b_labels.long())
        test_loss.append(loss.item())

        # get预测结果，这里就是求每行最大的索引咯，然后用flatten打平成一维
        preds = torch.argmax(logits, dim=1).flatten()  # 返回一行中最大值的序号

        # 计算准确率，这个就是俩比较，返回相同的个数, .cpu().numpy()就是把tensor从显卡上取出来然后转化为numpy类型的举证好用方法
        # 最后mean因为直接bool形了，也就是如果预测和label一样那就返回1，正好是正确的个数，求平均就是准确率了
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        test_accuracy.append(accuracy)

    # 计算整体的平均正确率和loss
    val_loss = np.mean(test_loss)
    val_accuracy = np.mean(test_accuracy)

    return val_loss, val_accuracy

In [15]:
# 先来个两轮

bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
# print("Start training and validation:\n")
print("Start training and testing:\n")
train(bert_classifier, train_dataloader, eval_dataloader, epochs=2, evaluation=True)  # 这个是有评估的


net = BertClassifier()
print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in net.parameters())))

initializing classifier!


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


to device!
initializing optimizer!
initializing total steps!
initializing scheduler!
Start training and testing:

 Epoch  | 每40个Batch |   训练集 Loss   |  测试集 Loss  |  测试集准确率   |    时间    
--------------------------------------------------------------------------------
   1    |     40     |    0.878029    |      -       |       -       |   24.62  
   1    |     80     |    0.653884    |      -       |       -       |   24.76  
   1    |    101     |    0.633879    |      -       |       -       |   12.74  
--------------------------------------------------------------------------------
   1    |     -      |    0.739863    |   0.672357   |    75.61    % |   64.52  
--------------------------------------------------------------------------------


 Epoch  | 每40个Batch |   训练集 Loss   |  测试集 Loss  |  测试集准确率   |    时间    
--------------------------------------------------------------------------------
   2    |     40     |    0.470694    |      -       |       -       |   25.14  
   2    |  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total number of paramerters in networks is 109559645  


In [26]:
# 预测测试集
test_inputs, test_masks = preprocessing_for_bert(test_texts, test_entities)
test_dataset = TensorDataset(test_inputs, test_masks)
test_dataloader = DataLoader(test_dataset, sampler=SequentialSampler(test_dataset), batch_size=BATCH_SIZE)

test_loss, test_accuracy = evaluate(bert_classifier, eval_dataloader)
print(test_loss, test_accuracy)

# 把模型放到测试模式
bert_classifier.eval()

# 预测
predictions = []
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask = batch
    with torch.no_grad():
        logits = bert_classifier(b_input_ids, b_input_mask)
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)

# 把预测结果转化为标签
pred_labels = []
for pred in predictions:
    pred_labels += list(np.argmax(pred, axis=1))

# 输出标签
print(pred_labels)
pred_labels = [(i-2) for i in pred_labels]
print(pred_labels)


0.5379325672984123 79.80324074074075
[3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 2, 2, 3, 3, 1, 3, 1, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 2, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 1, 1, 1, 2, 3, 3, 1, 3, 1, 3, 3, 3, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 2, 2,

In [27]:
with open('201300079_BERT.txt', 'w', encoding='utf-8') as f:
    for label in pred_labels:
        f.write(str(label) + '\n')